In [1]:
!pip install gdown
!pip install scikit-learn
!pip install dlib
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

import gdown
import os
import re
import cv2
import pandas as pd
import zipfile
import shutil
from google.colab import files
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import sys
import json
import dlib
import pickle
import matplotlib.pyplot as plt
from collections import defaultdict
import random

gdown.download_folder("https://drive.google.com/drive/folders/1xgp9CaRQgdIcJjkS2C8vLoMlgsBHyiXL?usp=sharing", quiet=True)

--2025-08-23 18:30:49--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-08-23 18:30:50--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  14.1MB/s    in 5.0s    

2025-08-23 18:30:56 (12.2 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



['/content/Miracl-VC1/dataset_miracl.zip',
 '/content/Miracl-VC1/lrw_resnet18_dctcn_video.pth']

In [2]:
DATASET_DIR = "/content/Miracl-VC1"
ANNOTATION_DIR = "/content/Lipreading_using_Temporal_Convolutional_Networks/annotations"
LABELS_TXT = "/content/labels.txt"
LABELS_DST = "/content/Lipreading_using_Temporal_Convolutional_Networks/labels/500WordsSortedList.txt"
MODEL_SRC = os.path.join(DATASET_DIR, 'lrw_resnet18_dctcn_video.pth')
MODEL_DST = '/content/Lipreading_using_Temporal_Convolutional_Networks/models/resnet18_dctcn_video.pth'

import os
import shutil
import zipfile

# ----------------------
# Estrai ZIP e pulizia
# ----------------------
def extract_and_remove_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)
    print(f"[ZIP] File zip {zip_path} estratto e rimosso.")

    # Rinomina le cartelle appena estratte in uppercase
    for item in os.listdir(extract_to):
        item_path = os.path.join(extract_to, item)
        if os.path.isdir(item_path):
            new_name = item.upper()
            new_path = os.path.join(extract_to, new_name)
            if new_name != item:
                os.rename(item_path, new_path)
                print(f"[RENAME] {item} → {new_name}")

# Estrazione zip
zip_files = [f for f in os.listdir(DATASET_DIR) if f.endswith('.zip')]
print(f"[ZIP] Zip files trovati: {zip_files}")
for zip_file in zip_files:
    extract_and_remove_zip(os.path.join(DATASET_DIR, zip_file), DATASET_DIR)

# ----------------------
# 🔥 Nuova parte: tieni solo la label START
# ----------------------
print("[CLEANUP] Mantengo solo la label START...")
for label in os.listdir(DATASET_DIR):
    label_path = os.path.join(DATASET_DIR, label)
    if os.path.isdir(label_path):
        if label.upper() != "START":
            shutil.rmtree(label_path)
            print(f"   → Eliminata cartella label: {label_path}")

video_path = "/content/Miracl-VC1/Choose/M01_09.mp4"

if os.path.exists(video_path):
    os.remove(video_path)
    print(f"[INFO] Video eliminato: {video_path}")
else:
    print(f"[WARNING] Il file non esiste: {video_path}")

# ----------------------
# Clone repo e modello
# ----------------------
!git clone https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks.git
!pip install -r Lipreading_using_Temporal_Convolutional_Networks/requirements.txt

import sys
sys.path.append('/content/Lipreading_using_Temporal_Convolutional_Networks')

# Sposta il modello .pth
os.makedirs(os.path.dirname(MODEL_DST), exist_ok=True)
if os.path.exists(MODEL_SRC):
    shutil.move(MODEL_SRC, MODEL_DST)
    print(f"[MODEL] Modello spostato in {MODEL_DST}")
else:
    print(f"[MODEL] ⚠️ Modello non trovato in {MODEL_SRC}")

# ----------------------
# Gestione labels
# ----------------------
labels = sorted([d for d in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, d))])
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Salva le label in labels.txt
with open(LABELS_TXT, "w") as f:
    for label in labels:
        f.write(label + "\n")
print(f"[LABELS] Salvate {len(labels)} label in {LABELS_TXT}")

# Fonde con le labels LRW se già presenti
lrw_labels = set()
if os.path.exists(LABELS_DST):
    with open(LABELS_DST, "r") as f:
        lrw_labels = set(line.strip().upper() for line in f if line.strip())

your_labels = set(label.upper() for label in labels)
merged_labels = sorted(lrw_labels.union(your_labels))

os.makedirs(os.path.dirname(LABELS_DST), exist_ok=True)
with open(LABELS_DST, "w") as f:
    for label in merged_labels:
        f.write(label + "\n")
print(f"[LABELS] File finale con {len(merged_labels)} label scritto in {LABELS_DST}")

import re
import random

DATASET_DIR = "/content/Miracl-VC1"
ANNOTATION_DIR = "/content/Lipreading_using_Temporal_Convolutional_Networks/annotations"
TEST_CSV = "/content/Lipreading_using_Temporal_Convolutional_Networks/test.csv"

def write_annotation_file(path, samples):
    with open(path, "w") as f:
        for rel_path, label_idx, speaker in samples:
            rel_path = rel_path.replace(".mp4", "")
            f.write(f"{rel_path} {label_idx} {speaker} 0 29\n")
    print(f"[ANNOTATION] Scritto {path} con {len(samples)} righe")

# === Crea mapping label → indice
labels = sorted([d for d in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, d))])
label_to_index = {label: idx for idx, label in enumerate(labels)}

# === Raccoglie tutti i campioni in un unico test set
all_samples = []
for label in labels:
    label_dir = os.path.join(DATASET_DIR, label)
    for fname in os.listdir(label_dir):
        if fname.endswith(".mp4"):
            speaker = fname.split("_")[0] if "_" in fname else "UNK"
            rel_path = f"{label}/{fname}"
            label_idx = label_to_index[label]
            all_samples.append((rel_path, label_idx, speaker))

# === Salva solo file per il test ===
os.makedirs(ANNOTATION_DIR, exist_ok=True)
write_annotation_file(os.path.join(ANNOTATION_DIR, "test.txt"), all_samples)

with open(TEST_CSV, "w") as f_out:
    for rel_path, _, _ in all_samples:
        rel_path = rel_path.replace(".mp4", "")
        f_out.write(f"{rel_path},0\n")
print(f"[CSV] Creato {TEST_CSV}")


[ZIP] Zip files trovati: ['dataset_miracl.zip']
[ZIP] File zip /content/Miracl-VC1/dataset_miracl.zip estratto e rimosso.
[RENAME] Previous → PREVIOUS
[RENAME] Start → START
[RENAME] Hello → HELLO
[RENAME] Navigation → NAVIGATION
[RENAME] Connection → CONNECTION
[RENAME] Well → WELL
[RENAME] Stop → STOP
[RENAME] Hegin → HEGIN
[RENAME] Next → NEXT
[RENAME] Choose → CHOOSE
[CLEANUP] Mantengo solo la label START...
   → Eliminata cartella label: /content/Miracl-VC1/HEGIN
   → Eliminata cartella label: /content/Miracl-VC1/STOP
   → Eliminata cartella label: /content/Miracl-VC1/CONNECTION
   → Eliminata cartella label: /content/Miracl-VC1/NAVIGATION
   → Eliminata cartella label: /content/Miracl-VC1/NEXT
   → Eliminata cartella label: /content/Miracl-VC1/PREVIOUS
   → Eliminata cartella label: /content/Miracl-VC1/CHOOSE
   → Eliminata cartella label: /content/Miracl-VC1/HELLO
   → Eliminata cartella label: /content/Miracl-VC1/WELL
[WARNING] Il file non esiste: /content/Miracl-VC1/Choose/M01

In [3]:
output_root = "/content/Miracl-VC1"

for folder in os.listdir(output_root):
    new_folder = folder.rstrip('.')
    if new_folder != folder:
        old_path = os.path.join(output_root, folder)
        new_path = os.path.join(output_root, new_folder)
        os.rename(old_path, new_path)
        print(f"[RENAME] Rinomino {old_path} -> {new_path}")

def clean_video_remove_no_faces(video_path):
    face_detector = dlib.get_frontal_face_detector()

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[ERROR] Non riesco ad aprire il video {video_path}")
        return

    # Parametri video originali
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")

    # Temporaneo: salva in un file temporaneo prima di sovrascrivere
    temp_path = video_path + ".tmp.mp4"
    out = cv2.VideoWriter(temp_path, fourcc, fps, (width, height))

    frame_idx = 0
    kept_frames = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray)

        if len(faces) == 0:
            print(f"[FRAME {frame_idx}] Nessun volto rilevato, frame scartato")
        else:
            out.write(frame)
            kept_frames += 1

        frame_idx += 1

    cap.release()
    out.release()

    print(f"[VIDEO CLEAN] Da {frame_idx} frame, ne ho tenuti {kept_frames}")

    # Sovrascrivo il file originale con il pulito
    os.replace(temp_path, video_path)
    print(f"[VIDEO CLEAN] Sovrascritto {video_path} con versione pulita")

def save_pts_file(points, path):
    with open(path, "w") as f:
        f.write("version: 1\nn_points: 68\n{\n")
        for (x, y) in points:
            f.write(f"{x:.3f} {y:.3f}\n")
        f.write("}\n")
    print(f"[PTS] Salvati landmarks in {path}")

def save_landmarks_to_pkl(all_landmarks, output_pkl):
    os.makedirs(os.path.dirname(output_pkl), exist_ok=True)
    with open(output_pkl, 'wb') as f:
        pickle.dump(all_landmarks, f)
    print(f"[PKL] Salvato file .pkl con {len(all_landmarks)} set di landmark in {output_pkl}")

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

input_dir = output_root
landmark_root = "/content/Lipreading_using_Temporal_Convolutional_Networks/landmarks"
os.makedirs(landmark_root, exist_ok=True)

print(f"[LANDMARKS] Inizio estrazione landmarks dai video in {input_dir}")

for label in tqdm(os.listdir(input_dir), desc="Labels"):
    label_path = os.path.join(input_dir, label)
    if not os.path.isdir(label_path):
        continue

    for video_file in os.listdir(label_path):
        video_path = os.path.join(label_path, video_file)
        video_name = os.path.splitext(video_file)[0]

        print(f"[VIDEO] Pulizia video: {video_path}")
        clean_video_remove_no_faces(video_path)  # ✅ PRIMA

        print(f"[VIDEO] Estrazione landmarks: {video_path}")
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"[VIDEO] Numero di frame stimato: {frame_count}")

        frame_idx = 0
        all_landmarks = []
        pts_dir = os.path.join(landmark_root, label, video_name)
        os.makedirs(pts_dir, exist_ok=True)

        while True:
            ret, frame = cap.read()
            if not ret:
                print(f"[VIDEO] Fine video {video_file} dopo {frame_idx} frame")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_detector(gray)

            if len(faces) == 0:
                print(f"[FRAME {frame_idx}] Nessun volto rilevato, frame SCARTATO")
                frame_idx += 1
                continue

            shape = landmark_predictor(gray, faces[0])
            points = [(shape.part(i).x, shape.part(i).y) for i in range(68)]
            out_path = os.path.join(pts_dir, f"frame_{frame_idx:03d}.pts")
            save_pts_file(points, out_path)
            all_landmarks.append(np.array(points))
            frame_idx += 1

        cap.release()

        pkl_output_path = os.path.join(landmark_root, label, f"{video_name}.pkl")
        save_landmarks_to_pkl(all_landmarks, pkl_output_path)


video_list = []
for root, dirs, files in os.walk(output_root):
    for file in files:
        if file.endswith('.mp4'):
            rel_path = os.path.relpath(os.path.join(root, file), output_root)
            video_list.append(rel_path)


[LANDMARKS] Inizio estrazione landmarks dai video in /content/Miracl-VC1


Labels:   0%|          | 0/1 [00:00<?, ?it/s]

[VIDEO] Pulizia video: /content/Miracl-VC1/START/F10_09.mp4
[VIDEO CLEAN] Da 8 frame, ne ho tenuti 8
[VIDEO CLEAN] Sovrascritto /content/Miracl-VC1/START/F10_09.mp4 con versione pulita
[VIDEO] Estrazione landmarks: /content/Miracl-VC1/START/F10_09.mp4
[VIDEO] Numero di frame stimato: 8
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F10_09/frame_000.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F10_09/frame_001.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F10_09/frame_002.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F10_09/frame_003.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F10_09/frame_004.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F1

Labels: 100%|██████████| 1/1 [02:54<00:00, 174.13s/it]

[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F11_04/frame_005.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F11_04/frame_006.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F11_04/frame_007.pts
[PTS] Salvati landmarks in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F11_04/frame_008.pts
[VIDEO] Fine video F11_04.mp4 dopo 9 frame
[PKL] Salvato file .pkl con 9 set di landmark in /content/Lipreading_using_Temporal_Convolutional_Networks/landmarks/START/F11_04.pkl


In [11]:
import urllib.request
import bz2
import shutil
import os
import cv2
import dlib
import numpy as np
import pickle
from tqdm import tqdm

dataloader_patch = '''

#! /usr/bin/env python
# -*- coding: utf-8 -*-

# Copyright 2021 Imperial College London (Pingchuan Ma)
# Apache 2.0  (http://www.apache.org/licenses/LICENSE-2.0)
import cv2
import pickle
import numpy as np
from transform import *
from utils import *


class AVSRDataLoader(object):

    def __init__(
        self,
        mean_face_path="20words_mean_face.npy",
        crop_width=96,
        crop_height=96,
        start_idx=48,
        stop_idx=68,
        window_margin=12,
        convert_gray=True):

        self._reference = np.load(os.path.join( os.path.dirname(__file__), mean_face_path))
        self._crop_width = crop_width
        self._crop_height = crop_height
        self._start_idx = start_idx
        self._stop_idx = stop_idx
        self._window_margin = window_margin
        self._convert_gray = convert_gray


    def preprocess(self, video_pathname, landmarks_pathname):
            print(f"[DEBUG][preprocess] Carico landmarks da: {landmarks_pathname}")

            if isinstance(landmarks_pathname, str):
                # Controlla se il file è vuoto
                if not os.path.exists(landmarks_pathname) or os.path.getsize(landmarks_pathname) == 0:
                    print(f"[ERROR] Il file dei landmarks non esiste o è vuoto: {landmarks_pathname}")
                    return None

                try:
                    with open(landmarks_pathname, "rb") as pf:
                        landmarks = pickle.load(pf)
                except (pickle.UnpicklingError, EOFError) as e:
                    print(f"[ERROR] Errore nel caricare il file pickle {landmarks_pathname}: {e}")
                    return None
            else:
                landmarks = landmarks_pathname

            num_none = sum(1 for l in landmarks if l is None)
            print(f"[DEBUG][preprocess] Landmarks caricati. Num frame: {len(landmarks)} | con None: {num_none}")

            preprocessed_landmarks = self.landmarks_interpolate(landmarks)
            none_after = sum(1 for l in preprocessed_landmarks if l is None) if preprocessed_landmarks is not None else 'N/A'
            print(f"[DEBUG] Dopo interpolazione: {len(preprocessed_landmarks)} landmarks | con None: {none_after}")

            cap = cv2.VideoCapture(video_pathname)
            if not cap.isOpened():
                print(f"[ERROR] Impossibile aprire il file video: {video_pathname}")
                return None
            video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
            print(f"[DEBUG] Frame video: {video_frames} | Frame landmarks: {len(preprocessed_landmarks)}")

            # Aggiunto un controllo per gestire il caso in cui l'interpolazione restituisce None
            if preprocessed_landmarks is None:
                print(f"[ERROR] Interpolazione dei landmarks fallita per {video_pathname}")
                return None

            if video_frames != len(preprocessed_landmarks):
                print(f"[ERROR][preprocess] Mismatch frame-video ({video_frames}) vs landmarks ({len(preprocessed_landmarks)})")
                return None

            n = len(preprocessed_landmarks)
            if n < self._window_margin:
                print(f"[WARNING] Solo {n} frame (< window_margin={self._window_margin}); procedo comunque.")

            sequence, _, _ = self.crop_patch(video_pathname, preprocessed_landmarks)

            if sequence is None or len(sequence) == 0:
                print(f"[ERROR] sequence None o vuota per {video_pathname}")
            else:
                print(f"[DEBUG] sequence generata con {len(sequence)} frame")

            return sequence


    def landmarks_interpolate(self, landmarks):
        print(f"[DEBUG][interpolate] Landmarks in input: {len(landmarks)}")
        valid_frames_idx = [idx for idx, _ in enumerate(landmarks) if _ is not None]

        if not valid_frames_idx:
            return None

        for idx in range(1, len(valid_frames_idx)):
            if valid_frames_idx[idx] - valid_frames_idx[idx-1] == 1:
                continue
            else:
                landmarks = linear_interpolate(landmarks, valid_frames_idx[idx-1], valid_frames_idx[idx])

        valid_frames_idx = [idx for idx, _ in enumerate(landmarks) if _ is not None]

        # -- Corner case: riempi inizio/fine con l’ultimo valido
        if valid_frames_idx:
            landmarks[:valid_frames_idx[0]] = [landmarks[valid_frames_idx[0]]] * valid_frames_idx[0]
            landmarks[valid_frames_idx[-1]:] = [landmarks[valid_frames_idx[-1]]] * (len(landmarks) - valid_frames_idx[-1])

        valid_frames_idx = [idx for idx, _ in enumerate(landmarks) if _ is not None]

        assert len(valid_frames_idx) == len(landmarks), "not every frame has landmark"

        print(f"[DEBUG][interpolate] Landmarks dopo interpolazione: {len(landmarks)}")
        return landmarks


    def crop_patch(self, video_pathname, landmarks):

        frame_idx = 0
        frame_gen = load_video(video_pathname)
        while True:
            if frame_idx >= len(landmarks):
              print(f"[ERROR] frame_idx {frame_idx} >= len(landmarks) {len(landmarks)} → desync video/landmark")
              break
            try:
                frame = frame_gen.__next__() ## -- BGR
            except StopIteration:
                break
            if frame_idx == 0:
                sequence = []

                sequence_frame = []
                sequence_landmarks = []
            window_margin = min(self._window_margin // 2, frame_idx, len(landmarks) - 1 - frame_idx)
            if window_margin < 0:
                window_margin = 0
            smoothed_landmarks = np.mean([landmarks[x] for x in range(frame_idx - window_margin, frame_idx + window_margin + 1)], axis=0)
            smoothed_landmarks += landmarks[frame_idx].mean(axis=0) - smoothed_landmarks.mean(axis=0)
            transformed_frame, transformed_landmarks = self.affine_transform(frame, smoothed_landmarks, self._reference, grayscale=self._convert_gray)
            sequence.append( cut_patch( transformed_frame,
                                        transformed_landmarks[self._start_idx:self._stop_idx],
                                        self._crop_height//2,
                                        self._crop_width//2,))

            sequence_frame.append( transformed_frame)
            sequence_landmarks.append( transformed_landmarks)

            frame_idx += 1
        return np.array(sequence), np.array(sequence_frame), np.array(sequence_landmarks)


    def affine_transform(self, frame, landmarks, reference, grayscale=False, target_size=(256, 256),
                         reference_size=(256, 256), stable_points=(28, 33, 36, 39, 42, 45, 48, 54),
                         interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT,
                         border_value=0):
        # Prepare everything
        if grayscale and frame.ndim == 3:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        stable_reference = np.vstack([reference[x] for x in stable_points])
        stable_reference[:, 0] -= (reference_size[0] - target_size[0]) / 2.0
        stable_reference[:, 1] -= (reference_size[1] - target_size[1]) / 2.0

        # Warp the face patch and the landmarks
        transform = cv2.estimateAffinePartial2D(np.vstack([landmarks[x] for x in stable_points]),
                                                stable_reference, method=cv2.LMEDS)[0]
        transformed_frame = cv2.warpAffine(frame, transform, dsize=(target_size[0], target_size[1]),
                                    flags=interpolation, borderMode=border_mode, borderValue=border_value)
        transformed_landmarks = np.matmul(landmarks, transform[:, :2].transpose()) + transform[:, 2].transpose()

        return transformed_frame, transformed_landmarks


    def load_audio(self, data_filename):
        sequence = load_audio(data_filename, specified_sr=16000, int_16=False)
        return sequence

    def load_video(self, data_filename, landmarks_filename=None):

        assert landmarks_filename is not None
        sequence = self.preprocess(
            video_pathname=data_filename,
            landmarks_pathname=landmarks_filename,
        )
        return sequence

    def load_data(self, modality, data_filename, landmarks_filename=None):
        if modality == "raw_audio":
            return self.load_audio(data_filename)
        elif modality == "video":
            return self.load_video(data_filename, landmarks_filename=landmarks_filename)
'''

with open("/content/Lipreading_using_Temporal_Convolutional_Networks/preprocessing/dataloader.py", "w", encoding='utf-8') as f:
    f.write(dataloader_patch)

print("dataloader.py patchato con successo.")


dataloaders_patch = """
import torch
import numpy as np
from lipreading.preprocess import *
from lipreading.dataset import MyDataset, pad_packed_collate


def get_preprocessing_pipelines(modality):
    # -- preprocess for the video stream
    preprocessing = {}
    # -- LRW config
    if modality == 'video':
        crop_size = (88, 88)
        (mean, std) = (0.421, 0.165)
        preprocessing['train'] = Compose([
                                    Normalize( 0.0,255.0 ),
                                    RandomCrop(crop_size),
                                    HorizontalFlip(0.5),
                                    Normalize(mean, std),
                                    TimeMask(T=0.6*25, n_mask=1)
                                    ])

        preprocessing['val'] = Compose([
                                    Normalize( 0.0,255.0 ),
                                    CenterCrop(crop_size),
                                    Normalize(mean, std) ])

        preprocessing['test'] = preprocessing['val']

    elif modality == 'audio':

        preprocessing['train'] = Compose([
                                    AddNoise( noise=np.load('./data/babbleNoise_resample_16K.npy')),
                                    NormalizeUtterance()])

        preprocessing['val'] = NormalizeUtterance()

        preprocessing['test'] = NormalizeUtterance()

    return preprocessing


def get_data_loaders(args):
    preprocessing = get_preprocessing_pipelines( args.modality)

    # create dataset object for each partition
    partitions = ['test'] if args.test else ['train', 'val', 'test']
    dsets = {partition: MyDataset(
                modality=args.modality,
                data_partition=partition,
                data_dir=args.data_dir,
                label_fp=args.label_path,
                labels=args.labels,
                annonation_direc=args.annonation_direc,
                preprocessing_func=preprocessing[partition],
                data_suffix='.npz',
                use_boundary=args.use_boundary,
                ) for partition in partitions}
    for partition in partitions:
        print(f"[DEBUG] Partition '{partition}': {len(dsets[partition])} samples")
    dset_loaders = {x: torch.utils.data.DataLoader(
                        dsets[x],
                        batch_size=args.batch_size,
                        shuffle=True,
                        collate_fn=pad_packed_collate,
                        pin_memory=True,
                        num_workers=args.workers,
                        worker_init_fn=np.random.seed(1)) for x in partitions}
    return dset_loaders

"""

with open("/content/Lipreading_using_Temporal_Convolutional_Networks/lipreading/dataloaders.py", "w", encoding='utf-8') as f:
    f.write(dataloaders_patch)

print("dataloaders.py patchato con successo.")

dataset_patch = '''

import os
import glob
import torch
import random
import librosa
import numpy as np
import sys
from lipreading.utils import read_txt_lines


class MyDataset(object):
    def __init__(self, data_partition, modality, data_dir, label_fp, labels, annonation_direc=None,
                 preprocessing_func=None, data_suffix='.npz', use_boundary=False):

        assert os.path.isfile(label_fp), \
            f"File path provided for the labels does not exist. Path input: {label_fp}."

        self.data_partition = data_partition
        self._data_dir = data_dir
        self._data_suffix = data_suffix
        self._label_fp = label_fp
        self.labels = labels
        print("[CONTROL]: ", labels)
        self._annonation_direc = annonation_direc

        self.fps = 25 if modality == "video" else 16000
        self.is_var_length = False
        self.use_boundary = use_boundary
        self.label_idx = -2
        self.preprocessing_func = preprocessing_func

        if self.use_boundary or (self.is_var_length):
            assert self._annonation_direc is not None, \
                "Directory path provided for the sequence timestamp (--annonation-direc) should not be empty."
            assert os.path.isdir(self._annonation_direc), \
                f"Directory path provided for the sequence timestamp (--annonation-direc) does not exist. Directory input: {self._annonation_direc}"

        print(f"[DEBUG][{modality}] Inizializzazione dataset")
        print(f"[DEBUG] data_dir = {self._data_dir}")
        print(f"[DEBUG] label_fp = {self._label_fp}")

        self.data = []

        # Aggiunta solo di log di debug: stampiamo cosa sta cercando di caricare
        with open(label_fp, "r") as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip()
            if not line: continue
            parts = line.split()
            rel_path = parts[0]  # solo il path relativo al file .mp4
            npz_path = os.path.join(self._data_dir, rel_path + self._data_suffix)
            if os.path.exists(npz_path):
                print(f"[FOUND] {npz_path}")
                self.data.append(npz_path)
            else:
                print(f"[MISSING] {npz_path}")

        print(f"[DEBUG] Trovati {len(self.data)} esempi validi per il partition")

        self._data_files = []
        self.load_dataset()

    def load_dataset(self):

        # -- read the labels file
        self._labels = read_txt_lines(self.labels)
        print("[DEBUG labels]", self._labels)


        # -- add examples to self._data_files
        self._get_files_for_partition()


        # -- from self._data_files to self.list
        self.list = dict()
        self.instance_ids = dict()
        for i, x in enumerate(self._data_files):
            label = self._get_label_from_path( x )
            print(f"[DEBUG] path: {x} → label estratta: {label}")
            self.list[i] = [ x, self._labels.index( label ) ]
            self.instance_ids[i] = self._get_instance_id_from_path( x )

    def _get_instance_id_from_path(self, x):
        # for now this works for npz/npys, might break for image folders
        # --- INIZIO CORREZIONE 1/3
        instance_id = os.path.basename(x)
        return os.path.splitext( instance_id )[0]
        # --- FINE CORREZIONE 1/3

    def _get_label_from_path(self, x):
        # --- INIZIO CORREZIONE 2/3
        return os.path.basename(os.path.dirname(x))
        # --- FINE CORREZIONE 2/3

    def _get_files_for_partition(self):
        # Cerca i file npz/npy/mp4 in tutte le sottocartelle (label)
        dir_fp = self._data_dir
        if not dir_fp:
            print("[ERRORE] data_dir è None o vuota!")
            return

        # Utilizza glob.glob che è già compatibile con entrambi i separatori
        search_str_npz = os.path.join(dir_fp, '*', '*.npz')
        search_str_npy = os.path.join(dir_fp, '*', '*.npy')
        search_str_mp4 = os.path.join(dir_fp, '*', '*.mp4')

        self._data_files.extend(glob.glob(search_str_npz))
        self._data_files.extend(glob.glob(search_str_npy))
        self._data_files.extend(glob.glob(search_str_mp4))

        print(f"[DEBUG] Trovati {len(self._data_files)} file prima del filtraggio")
        print("[DEBUG] Prime 5 path trovate:")
        for f in self._data_files[:5]:
            print(f"  - {f}")

        # Mostra le label attese caricate dal file
        print(f"[DEBUG] Numero totale di label attese: {len(self._labels)}")
        print("[DEBUG] Esempi di label attese:")
        for lbl in self._labels[:10]:
            print(f"  - '{lbl}'")

        match_count = 0
        mismatch_count = 0

        # Analizza tutti i file e diagnostica il motivo del filtraggio
        for f in self._data_files:
            # Usa os.path.normpath per uniformare i separatori e os.path.split per estrarre il nome della directory
            parent_dir = os.path.normpath(f).split(os.path.sep)[-2]
            label_match = parent_dir in self._labels

            if label_match:
                match_count += 1
            else:
                mismatch_count += 1
                print(f"[MISMATCH] Directory '{parent_dir}' NON trovata tra le label attese → {f}")

        # Effettua il filtraggio
        self._data_files = [
            f for f in self._data_files
            if os.path.normpath(f).split(os.path.sep)[-2] in self._labels
        ]

        print(f"[DEBUG] Trovati {len(self._data_files)} file dopo il filtraggio per label")
        print(f"[DEBUG] Totale match: {match_count}, mismatch: {mismatch_count}")

        # Esempi finali dei file mantenuti
        print("[DEBUG] Esempi di file accettati dopo filtraggio:")
        for f in self._data_files[:5]:
            print(f"  - {f}")

        # Diagnostica avanzata: se 0 file trovati, controlla case sensitivity
        if match_count == 0:
            print("[POSSIBILE PROBLEMA] Nessuna directory trovata tra le label!")
            print("Verifica se le label hanno problemi di maiuscole/minuscole.")
            print("Esempio confronto tra directory e label attese:")
            example_dir_names = set([os.path.normpath(f).split(os.path.sep)[-2] for f in self._data_files])
            for example in list(example_dir_names)[:5]:
                in_labels = example in self._labels
                in_labels_lower = example.lower() in [l.lower() for l in self._labels]
                print(f"  '{example}' in labels? → {in_labels}")
                print(f"  '{example.lower()}' in labels.lower()? → {in_labels_lower}")


    def _apply_variable_length_aug(self, filename, raw_data):
        # read info txt file (to see duration of word, to be used to do temporal cropping)
        # --- INIZIO CORREZIONE 3/3
        # L'uso di `*filename.split('/')` non è compatibile con Windows
        rel_path_parts = filename.split(os.path.sep)[self.label_idx:]
        info_txt = os.path.join(self._annonation_direc, *rel_path_parts)
        # --- FINE CORREZIONE 3/3

        info_txt = os.path.splitext( info_txt )[0] + '.txt'  # swap extension
        info = read_txt_lines(info_txt)

        utterance_duration = float( info[4].split(' ')[1] )
        half_interval = int( utterance_duration/2.0 * self.fps)  # num frames of utterance / 2

        n_frames = raw_data.shape[0]
        mid_idx = ( n_frames -1 ) // 2  # video has n frames, mid point is (n-1)//2 as count starts with 0
        left_idx = random.randint(0, max(0,mid_idx-half_interval-1)  )  # random.randint(a,b) chooses in [a,b]
        right_idx = random.randint( min( mid_idx+half_interval+1,n_frames ), n_frames  )

        return raw_data[left_idx:right_idx]


    def _get_boundary(self, filename, raw_data):
        # read info txt file (to see duration of word, to be used to do temporal cropping)
        # --- INIZIO CORREZIONE 3/3
        # L'uso di `*filename.split('/')` non è compatibile con Windows
        rel_path_parts = filename.split(os.path.sep)[self.label_idx:]
        info_txt = os.path.join(self._annonation_direc, *rel_path_parts)
        # --- FINE CORREZIONE 3/3

        info_txt = os.path.splitext( info_txt )[0] + '.txt'  # swap extension
        info = read_txt_lines(info_txt)

        utterance_duration = float( info[4].split(' ')[1] )
        # boundary is used for the features at the top of ResNet, which as a frame rate of 25fps.
        if self.fps == 25:
            half_interval = int( utterance_duration/2.0 * self.fps)
            n_frames = raw_data.shape[0]
        elif self.fps == 16000:
            half_interval = int( utterance_duration/2.0 * 25)
            n_frames = raw_data.shape[0] // 640

        mid_idx = ( n_frames -1 ) // 2  # video has n frames, mid point is (n-1)//2 as count starts with 0
        left_idx = max(0, mid_idx-half_interval-1)
        right_idx = min(mid_idx+half_interval+1, n_frames)

        boundary = np.zeros(n_frames)
        boundary[left_idx:right_idx] = 1
        return boundary

    def load_data(self, path):
        arr = np.load(path)
        return arr['data'] if 'data' in arr else arr

    def __getitem__(self, idx):
        path, label = self.list[idx]
        print(f"[__getitem__] Loading sample {idx} | Path: {path} | Label: {label}")

        try:
            raw = self.load_data(path)
            if raw.size == 0:
                raise ValueError("Loaded data is empty.")
        except Exception as e:
            # Gestisce il caso di file .npz vuoti o corrotti
            print(f"[ERROR] Impossibile caricare il file {path}: {e}")
            return None, None

        # Se è RGB, converti in scala di grigio
        if raw.ndim == 4 and raw.shape[-1] == 3:
            r, g, b = raw[..., 0], raw[..., 1], raw[..., 2]
            raw = 0.2989*r + 0.5870*g + 0.1140*b  # (T, H, W)
            print(f"[__getitem__] Converted to grayscale: shape = {raw.shape}")

        if self.is_var_length and not self.use_boundary:
            raw = self._apply_variable_length_aug(path, raw)
            print(f"[__getitem__] Applied variable length augmentation.")

        data = self.preprocessing_func(raw) if self.preprocessing_func else raw
        print(f"[__getitem__] Preprocessed data shape: {data.shape}")

        if self.use_boundary:
            boundary = self._get_boundary(path, raw)
            print(f"[__getitem__] Boundary shape: {boundary.shape}")
            return data, label, boundary

        return data, label


    def __len__(self):
        return len(self._data_files)


def pad_packed_collate(batch):
    # Filtra i campioni che sono stati saltati (__getitem__ ha restituito None)
    batch = [item for item in batch if item[0] is not None]

    if not batch:
        # Restituisci tensori vuoti per evitare errori se l'intero batch è corrotto
        return torch.FloatTensor([]), [], torch.LongTensor([]), None

    if len(batch[0]) == 2:
        use_boundary = False
        data_tuple, lengths, labels_tuple = zip(*[(a, a.shape[0], b) for (a, b) in sorted(batch, key=lambda x: x[0].shape[0], reverse=True)])
    elif len(batch[0]) == 3:
        use_boundary = True
        data_tuple, lengths, labels_tuple, boundaries_tuple = zip(*[(a, a.shape[0], b, c) for (a, b, c) in sorted(batch, key=lambda x: x[0].shape[0], reverse=True)])

    if data_tuple[0].ndim == 1:
        max_len = data_tuple[0].shape[0]
        data_np = np.zeros((len(data_tuple), max_len))
    elif data_tuple[0].ndim == 3:
        max_len, h, w = data_tuple[0].shape
        data_np = np.zeros((len(data_tuple), max_len, h, w))
    for idx in range( len(data_np)):
        data_np[idx][:data_tuple[idx].shape[0]] = data_tuple[idx]
    data = torch.FloatTensor(data_np)

    if use_boundary:
        boundaries_np = np.zeros((len(boundaries_tuple), len(boundaries_tuple[0])))
        for idx in range(len(data_np)):
            boundaries_np[idx] = boundaries_tuple[idx]
        boundaries = torch.FloatTensor(boundaries_np).unsqueeze(-1)

    labels = torch.LongTensor(labels_tuple)

    if use_boundary:
        return data, lengths, labels, boundaries
    else:
        return data, lengths, labels

'''

with open("/content/Lipreading_using_Temporal_Convolutional_Networks/lipreading/dataset.py", "w", encoding='utf-8') as f:
    f.write(dataset_patch)

print("Dataset sovrascritto con successo.")


new_main = '''

#! /usr/bin/env python
# -*- coding: utf-8 -*-

# Copyright 2020 Imperial College London (Pingchuan Ma)
# Apache 2.0  (http://www.apache.org/licenses/LICENSE-2.0)

""" TCN for lipreading"""

import os
import time
import random
import argparse
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from lipreading.utils import get_save_folder
from lipreading.utils import load_json, save2npz
from lipreading.utils import load_model, CheckpointSaver
from lipreading.utils import get_logger, update_logger_batch
from lipreading.utils import showLR, calculateNorm2, AverageMeter
from lipreading.model import Lipreading
from lipreading.mixup import mixup_data, mixup_criterion
from lipreading.optim_utils import get_optimizer, CosineScheduler
from lipreading.dataloaders import get_data_loaders, get_preprocessing_pipelines


def load_args(default_config=None):
    parser = argparse.ArgumentParser(description='Pytorch Lipreading ')
    # -- dataset config
    parser.add_argument('--dataset', default='lrw', help='dataset selection')
    parser.add_argument('--num-classes', type=int, default=500, help='Number of classes')
    parser.add_argument('--modality', default='video', choices=['video', 'audio'], help='choose the modality')
    # -- directory
    parser.add_argument('--data-dir', default='./datasets/LRW_h96w96_mouth_crop_gray', help='Loaded data directory')
    parser.add_argument('--label-path', type=str, default='./labels/500WordsSortedList.txt', help='Path to txt file with paths to visual_data')
    parser.add_argument('--labels', type=str, default='./labels/500WordsSortedList.txt', help='Path to txt file with labels')
    parser.add_argument('--annonation-direc', default=None, help='Loaded data directory')
    # -- model config
    parser.add_argument('--backbone-type', type=str, default='resnet', choices=['resnet', 'shufflenet'], help='Architecture used for backbone')
    parser.add_argument('--relu-type', type=str, default='relu', choices=['relu','prelu'], help='what relu to use' )
    parser.add_argument('--width-mult', type=float, default=1.0, help='Width multiplier for mobilenets and shufflenets')
    # -- TCN config
    parser.add_argument('--tcn-kernel-size', type=int, nargs="+", help='Kernel to be used for the TCN module')
    parser.add_argument('--tcn-num-layers', type=int, default=4, help='Number of layers on the TCN module')
    parser.add_argument('--tcn-dropout', type=float, default=0.2, help='Dropout value for the TCN module')
    parser.add_argument('--tcn-dwpw', default=False, action='store_true', help='If True, use the depthwise seperable convolution in TCN architecture')
    parser.add_argument('--tcn-width-mult', type=int, default=1, help='TCN width multiplier')
    # -- DenseTCN config
    parser.add_argument('--densetcn-block-config', type=int, nargs = "+", help='number of denselayer for each denseTCN block')
    parser.add_argument('--densetcn-kernel-size-set', type=int, nargs = "+", help='kernel size set for each denseTCN block')
    parser.add_argument('--densetcn-dilation-size-set', type=int, nargs = "+", help='dilation size set for each denseTCN block')
    parser.add_argument('--densetcn-growth-rate-set', type=int, nargs = "+", help='growth rate for DenseTCN')
    parser.add_argument('--densetcn-dropout', default=0.2, type=float, help='Dropout value for DenseTCN')
    parser.add_argument('--densetcn-reduced-size', default=256, type=int, help='the feature dim for the output of reduce layer')
    parser.add_argument('--densetcn-se', default = False, action='store_true', help='If True, enable SE in DenseTCN')
    parser.add_argument('--densetcn-condense', default = False, action='store_true', help='If True, enable condenseTCN')
    # -- train
    parser.add_argument('--training-mode', default='tcn', help='tcn')
    parser.add_argument('--batch-size', type=int, default=32, help='Mini-batch size')
    parser.add_argument('--optimizer',type=str, default='adamw', choices = ['adam','sgd','adamw'])
    parser.add_argument('--lr', default=3e-4, type=float, help='initial learning rate')
    parser.add_argument('--init-epoch', default=0, type=int, help='epoch to start at')
    parser.add_argument('--epochs', default=80, type=int, help='number of epochs')
    parser.add_argument('--test', default=False, action='store_true', help='training mode')
    # -- mixup
    parser.add_argument('--alpha', default=0.4, type=float, help='interpolation strength (uniform=1., ERM=0.)')
    # -- test
    parser.add_argument('--model-path', type=str, default=None, help='Pretrained model pathname')
    parser.add_argument('--allow-size-mismatch', default=False, action='store_true',
                         help='If True, allows to init from model with mismatching weight tensors. Useful to init from model with diff. number of classes')
    # -- feature extractor
    parser.add_argument('--extract-feats', default=False, action='store_true', help='Feature extractor')
    parser.add_argument('--mouth-patch-path', type=str, default=None, help='Path to the mouth ROIs, assuming the file is saved as numpy.array')
    parser.add_argument('--mouth-embedding-out-path', type=str, default=None, help='Save mouth embeddings to a specificed path')
    # -- json pathname
    parser.add_argument('--config-path', type=str, default=None, help='Model configuration with json format')
    # -- other vars
    parser.add_argument('--interval', default=50, type=int, help='display interval')
    parser.add_argument('--workers', default=8, type=int, help='number of data loading workers')
    # paths
    parser.add_argument('--logging-dir', type=str, default='./train_logs', help = 'path to the directory in which to save the log file')
    # use boundaries
    parser.add_argument('--use-boundary', default=False, action='store_true', help='include hard border at the testing stage.')

    args = parser.parse_args()
    return args


args = load_args()

torch.manual_seed(1)
np.random.seed(1)
random.seed(1)
torch.backends.cudnn.benchmark = True

# Seleziona il dispositivo (CPU)
device = torch.device("cpu")

def extract_feats(model):
    """
    :rtype: FloatTensor
    """
    model.eval()
    preprocessing_func = get_preprocessing_pipelines()['test']
    data = preprocessing_func(np.load(args.mouth_patch_path)['data'])  # data: TxHxW
    return model(torch.FloatTensor(data)[None, None, :, :, :].to(device), lengths=[data.shape[0]])


def evaluate(model, dset_loader, criterion, idx_to_word=None):
    model.eval()
    running_loss = 0.
    running_corrects = 0.

    with torch.no_grad():
        for batch_idx, data in enumerate(tqdm(dset_loader)):
            if args.use_boundary:
                input, lengths, labels, boundaries = data
                boundaries = boundaries.to(device)
            else:
                input, lengths, labels = data
                boundaries = None

            input = input.unsqueeze(1).to(device)
            labels = labels.to(device)

            logits = model(input, lengths=lengths, boundaries=boundaries)
            _, preds = torch.max(F.softmax(logits, dim=1).data, dim=1)

            running_corrects += preds.eq(labels.view_as(preds)).sum().item()
            loss = criterion(logits, labels)
            running_loss += loss.item() * input.size(0)

            if idx_to_word:
                for i in range(len(labels)):
                    true_idx = labels[i].item()
                    pred_idx = preds[i].item()

                    true_word = idx_to_word.get(true_idx)
                    pred_word = idx_to_word.get(pred_idx)

                    if true_word is None:
                        print(f"[WARN] Missing GT label: {true_idx}")
                    if pred_word is None:
                        print(f"[WARN] Missing PRED label: {pred_idx}")

                    true_word = true_word if true_word else f"<UNK:{true_idx}>"
                    pred_word = pred_word if pred_word else f"<UNK:{pred_idx}>"
                    print(f"[EVAL] Sample {i:04d} | GT: {true_word}  | Pred: {pred_word}  {'✅' if true_idx == pred_idx else '❌'}")

    total = len(dset_loader.dataset)
    acc = running_corrects / total
    avg_loss = running_loss / total
    print(f"[EVAL] {total} samples total\tAccuracy: {acc:.4f}\tAvg Loss: {avg_loss:.4f}")
    return acc, avg_loss


def train(model, dset_loader, criterion, epoch, optimizer, logger):
    data_time = AverageMeter()
    batch_time = AverageMeter()

    lr = showLR(optimizer)

    logger.info('-' * 10)
    logger.info(f"Epoch {epoch}/{args.epochs - 1}")
    logger.info(f"Current learning rate: {lr}")

    model.train()
    running_loss = 0.
    running_corrects = 0.
    running_all = 0.

    end = time.time()
    for batch_idx, data in enumerate(dset_loader):
        if args.use_boundary:
            input, lengths, labels, boundaries = data
            boundaries = boundaries.to(device)
        else:
            input, lengths, labels = data
            boundaries = None
        data_time.update(time.time() - end)

        input, labels_a, labels_b, lam = mixup_data(input, labels, args.alpha)
        labels_a, labels_b = labels_a.to(device), labels_b.to(device)

        optimizer.zero_grad()

        logits = model(input.unsqueeze(1).to(device), lengths=lengths, boundaries=boundaries)

        loss_func = mixup_criterion(labels_a, labels_b, lam)
        loss = loss_func(criterion, logits)

        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()
        _, predicted = torch.max(F.softmax(logits, dim=1).data, dim=1)
        running_loss += loss.item()*input.size(0)
        running_corrects += lam * predicted.eq(labels_a.view_as(predicted)).sum().item() + (1 - lam) * predicted.eq(labels_b.view_as(predicted)).sum().item()
        running_all += input.size(0)
        if batch_idx % args.interval == 0 or (batch_idx == len(dset_loader)-1):
            update_logger_batch( args, logger, dset_loader, batch_idx, running_loss, running_corrects, running_all, batch_time, data_time )

    return model


def get_model_from_json():
    assert args.config_path.endswith('.json') and os.path.isfile(args.config_path), \
        f"'.json' config path does not exist. Path input: {args.config_path}"
    args_loaded = load_json( args.config_path)
    args.backbone_type = args_loaded['backbone_type']
    args.width_mult = args_loaded['width_mult']
    args.relu_type = args_loaded['relu_type']
    args.use_boundary = args_loaded.get("use_boundary", False)

    if args_loaded.get('tcn_num_layers', ''):
        tcn_options = { 'num_layers': args_loaded['tcn_num_layers'],
                        'kernel_size': args_loaded['tcn_kernel_size'],
                        'dropout': args_loaded['tcn_dropout'],
                        'dwpw': args_loaded['tcn_dwpw'],
                        'width_mult': args_loaded['tcn_width_mult'],
                      }
    else:
        tcn_options = {}
    if args_loaded.get('densetcn_block_config', ''):
        densetcn_options = {'block_config': args_loaded['densetcn_block_config'],
                            'growth_rate_set': args_loaded['densetcn_growth_rate_set'],
                            'reduced_size': args_loaded['densetcn_reduced_size'],
                            'kernel_size_set': args_loaded['densetcn_kernel_size_set'],
                            'dilation_size_set': args_loaded['densetcn_dilation_size_set'],
                            'squeeze_excitation': args_loaded['densetcn_se'],
                            'dropout': args_loaded['densetcn_dropout'],
                            }
    else:
        densetcn_options = {}

    model = Lipreading( modality=args.modality,
                        num_classes=args.num_classes,
                        tcn_options=tcn_options,
                        densetcn_options=densetcn_options,
                        backbone_type=args.backbone_type,
                        relu_type=args.relu_type,
                        width_mult=args.width_mult,
                        use_boundary=args.use_boundary,
                        extract_feats=args.extract_feats)

    calculateNorm2(model)
    return model


def get_label_mappings(npz_dir):
    labels = sorted([d for d in os.listdir(npz_dir) if os.path.isdir(os.path.join(npz_dir, d))])
    idx_to_word = {i: label for i, label in enumerate(labels)}
    word_to_idx = {label: i for i, label in enumerate(labels)}
    return idx_to_word, word_to_idx

def get_label_mappings_from_file(label_file_path):
    idx_to_word = {}
    word_to_idx = {}
    with open(label_file_path, 'r') as f:
        for idx, line in enumerate(f):
            word = line.strip()
            if word:  # Skip blank lines
                idx_to_word[idx] = word
                word_to_idx[word] = idx
    return idx_to_word, word_to_idx

def main():

    # -- logging
    save_path = get_save_folder(args)
    print(f"Model and log being saved in: {save_path}")
    logger = get_logger(args, save_path)
    ckpt_saver = CheckpointSaver(save_path)

    # -- get model
    model = get_model_from_json()

    # Sposta il modello sulla CPU qui, dopo l'inizializzazione
    model = model.to(device)

    # -- get dataset iterators
    dset_loaders = get_data_loaders(args)

    # -- get loss function
    criterion = nn.CrossEntropyLoss()

    # -- get optimizer
    optimizer = get_optimizer(args, optim_policies=model.parameters())

    # -- get learning rate scheduler
    scheduler = CosineScheduler(args.lr, args.epochs)

    # -- get label mappings
    label_file = "/content/Lipreading_using_Temporal_Convolutional_Networks/labels/500WordsSortedList.txt"
    idx_to_word, word_to_idx = get_label_mappings_from_file(label_file)
    print(f"[DEBUG] Loaded {len(idx_to_word)} labels in idx_to_word")

    # -- model resume or load
    if args.model_path:
        assert os.path.isfile(args.model_path), f"Model path does not exist: {args.model_path}"

        # Gestione del caricamento del modello
        if args.init_epoch > 0:
            # Caricamento di un checkpoint completo (per riprendere il training)
            model, optimizer, epoch_idx, ckpt_dict = load_model(args.model_path, model, optimizer)
            args.init_epoch = epoch_idx
            ckpt_saver.set_best_from_ckpt(ckpt_dict)
            logger.info(f"Model and states loaded from {args.model_path}")
        else:
            # Caricamento solo dei pesi del modello (per il testing)
            model, _, _, _ = load_model(args.model_path, model, allow_size_mismatch=args.allow_size_mismatch)
            logger.info(f"Model loaded from {args.model_path}")

        # Sposta il modello sul dispositivo corretto dopo il caricamento
        model = model.to(device)

        if args.mouth_patch_path:
            save2npz(args.mouth_embedding_out_path, data=extract_feats(model).cpu().detach().numpy())
            return

        if args.test:
            acc_avg_test, loss_avg_test = evaluate(model, dset_loaders['test'], criterion, idx_to_word=idx_to_word)
            logger.info(f"[TEST] Loss: {loss_avg_test:.4f}\tAccuracy: {acc_avg_test:.4f}")
            return

    # -- fix learning rate after checkpoint resume
    if args.model_path and args.init_epoch > 0:
        scheduler.adjust_lr(optimizer, args.init_epoch - 1)

    epoch = args.init_epoch

    while epoch < args.epochs:
        model = train(model, dset_loaders['train'], criterion, epoch, optimizer, logger)
        acc_avg_val, loss_avg_val = evaluate(model, dset_loaders['val'], criterion, idx_to_word=idx_to_word)
        logger.info(f"[VAL] Epoch {epoch} | Loss: {loss_avg_val:.4f} | Accuracy: {acc_avg_val:.4f} | LR: {showLR(optimizer)}")

        # -- save checkpoint
        save_dict = {
            'epoch_idx': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }
        ckpt_saver.save(save_dict, acc_avg_val)
        scheduler.adjust_lr(optimizer, epoch)
        epoch += 1

    # -- final evaluation
    best_fp = os.path.join(ckpt_saver.save_dir, ckpt_saver.best_fn)
    # Carica il modello migliore salvato per l'ultima valutazione
    _ = load_model(best_fp, model)
    model = model.to(device)
    acc_avg_test, loss_avg_test = evaluate(model, dset_loaders['test'], criterion, idx_to_word=idx_to_word)
    logger.info(f"[BEST TEST] Loss: {loss_avg_test:.4f}\tAccuracy: {acc_avg_test:.4f}")



if __name__ == '__main__':
    main()
'''

with open("/content/Lipreading_using_Temporal_Convolutional_Networks/main.py", "w", encoding='utf-8') as f:
    f.write(new_main)

print("Main sovrascritto con successo.")


utils = '''
import os
import json
import numpy as np

import datetime
import logging

import json
import torch
import shutil


def calculateNorm2(model):
    para_norm = 0.
    for p in model.parameters():
        para_norm += p.data.norm(2)
    print('2-norm of the neural network: {:.4f}'.format(para_norm**.5))


def showLR(optimizer):
    return optimizer.param_groups[0]['lr']


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


# -- IO utils
def read_txt_lines(filepath):
    assert os.path.isfile( filepath ), "Error when trying to read txt file, path does not exist: {}".format(filepath)
    with open( filepath ) as myfile:
        content = myfile.read().splitlines()
    return content


def save_as_json(d, filepath):
    with open(filepath, 'w') as outfile:
        json.dump(d, outfile, indent=4, sort_keys=True)


def load_json( json_fp ):
    assert os.path.isfile( json_fp ), "Error loading JSON. File provided does not exist, cannot read: {}".format( json_fp )
    with open( json_fp, 'r' ) as f:
        json_content = json.load(f)
    return json_content


def save2npz(filename, data=None):
    assert data is not None, "data is {}".format(data)
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    np.savez_compressed(filename, data=data)


# -- checkpoints
class CheckpointSaver:
    def __init__(self, save_dir, checkpoint_fn='ckpt.pth', best_fn='ckpt.best.pth', best_step_fn='ckpt.best.step{}.pth', save_best_step=False, lr_steps=[]):
        """
        Only mandatory: save_dir
            Can configure naming of checkpoint files through checkpoint_fn, best_fn and best_stage_fn
            If you want to keep best-performing checkpoint per step
        """

        self.save_dir = save_dir

        # checkpoint names
        self.checkpoint_fn = checkpoint_fn
        self.best_fn = best_fn
        self.best_step_fn = best_step_fn

        # save best per step?
        self.save_best_step = save_best_step
        self.lr_steps = []

        # init var to keep track of best performing checkpoint
        self.current_best = 0

        # save best at each step?
        if self.save_best_step:
            assert lr_steps != [], "Since save_best_step=True, need proper value for lr_steps. Current: {}".format(lr_steps)
            self.best_for_stage = [0]*(len(lr_steps)+1)

    def save(self, save_dict, current_perf, epoch=-1):
        """
            Save checkpoint and keeps copy if current perf is best overall or [optional] best for current LR step
        """

        # save last checkpoint
        checkpoint_fp = os.path.join(self.save_dir, self.checkpoint_fn)

        # keep track of best model
        self.is_best = current_perf > self.current_best
        if self.is_best:
            self.current_best = current_perf
            best_fp = os.path.join(self.save_dir, self.best_fn)
        save_dict['best_prec'] = self.current_best

        # keep track of best-performing model per step [optional]
        if self.save_best_step:

            assert epoch >= 0, "Since save_best_step=True, need proper value for 'epoch'. Current: {}".format(epoch)
            s_idx = sum( epoch >= l for l in lr_steps )
            self.is_best_for_stage = current_perf > self.best_for_stage[s_idx]

            if self.is_best_for_stage:
                self.best_for_stage[s_idx] = current_perf
                best_stage_fp = os.path.join(self.save_dir, self.best_stage_fn.format(s_idx))
            save_dict['best_prec_per_stage'] = self.best_for_stage

        # save
        torch.save(save_dict, checkpoint_fp)
        print("Checkpoint saved at {}".format(checkpoint_fp))
        if self.is_best:
            shutil.copyfile(checkpoint_fp, best_fp)
        if self.save_best_step and self.is_best_for_stage:
            shutil.copyfile(checkpoint_fp, best_stage_fp)


    def set_best_from_ckpt(self, ckpt_dict):
        self.current_best = ckpt_dict['best_prec']
        self.best_for_stage = ckpt_dict.get('best_prec_per_stage',None)


def load_model(load_path, model, optimizer=None, device='cpu', allow_size_mismatch=False):
    if not os.path.isfile(load_path):
        raise FileNotFoundError(f"File does not exist: {load_path}")

    checkpoint = torch.load(load_path, map_location=torch.device('cpu'))

    if isinstance(checkpoint, dict):
        # This handles both simple state dicts and full checkpoints
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
        else:
            state_dict = checkpoint

        model.load_state_dict(state_dict, strict=not allow_size_mismatch)

        # Only load optimizer and epoch if they exist
        if optimizer and 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        # Return different values based on whether the full checkpoint is available
        if 'epoch_idx' in checkpoint:
            return model, optimizer, checkpoint['epoch_idx'], checkpoint
        else:
            # Handle cases where the checkpoint is just a model state dict
            return model, optimizer, -1, None

    else:
        # If the checkpoint is a simple model state dict without a dictionary wrapper
        model.load_state_dict(checkpoint, strict=not allow_size_mismatch)
        return model, optimizer, -1, None


# -- logging utils
def get_logger(args,save_path):
    log_path = '{}/{}_{}_{}classes_log.txt'.format(save_path,args.training_mode,args.lr,args.num_classes)
    logger = logging.getLogger("mylog")
    logger.setLevel(logging.INFO)
    fh = logging.FileHandler(log_path)
    fh.setLevel(logging.INFO)
    logger.addHandler(fh)
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    logger.addHandler(console)
    return logger


def update_logger_batch( args, logger, dset_loader, batch_idx, running_loss, running_corrects, running_all, batch_time, data_time ):
    perc_epoch = 100. * batch_idx / (len(dset_loader)-1)
    logger.info(f"[{running_all:5.0f}/{len(dset_loader.dataset):5.0f} ({perc_epoch:.0f}%)]\tLoss: {running_loss / running_all:.4f}\tAcc:{running_corrects / running_all:.4f}\tCost time:{batch_time.val:1.3f} ({batch_time.avg:1.3f})s\tData time:{data_time.val:1.3f} ({data_time.avg:1.3f})\tInstances per second: {args.batch_size/batch_time.avg:.2f}")

def get_save_folder( args):
    # create save and log folder
    save_path = '{}/{}'.format( args.logging_dir, args.training_mode )

    # Generate a timestamp and replace colons with hyphens for Windows compatibility
    timestamp = datetime.datetime.now().isoformat().split('.')[0].replace(':', '-')
    save_path += '/' + timestamp

    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    return save_path

'''

with open("/content/Lipreading_using_Temporal_Convolutional_Networks/lipreading/utils.py", "w", encoding='utf-8') as f:
    f.write(utils)

print("Utils sovrascritto con successo.")

dataloader.py patchato con successo.
dataloaders.py patchato con successo.
Dataset sovrascritto con successo.
Main sovrascritto con successo.
Utils sovrascritto con successo.


In [8]:
ANN_DIR = '/content/Lipreading_using_Temporal_Convolutional_Networks/annonations'
VIDEO_DIR = '/content/Miracl-VC1/'

for label in os.listdir(VIDEO_DIR):
    label_dir = os.path.join(VIDEO_DIR, label)
    ann_label_dir = os.path.join(ANN_DIR, label)
    os.makedirs(ann_label_dir, exist_ok=True)

    for npz in os.listdir(label_dir):
        video_name = os.path.splitext(npz)[0]
        video_path = os.path.join(VIDEO_DIR, label, video_name + '.mp4')
        if not os.path.isfile(video_path):
            continue
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        start, end = 0, frame_count - 1
        txt_path = os.path.join(ann_label_dir, f'{video_name}.txt')
        with open(txt_path, 'w') as f:
            f.write(f"{start} {end}")
        print(f"Created boundary file: {txt_path}")

Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/F10_09.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/F09_07.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/M08_09.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/M08_01.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/F07_04.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/F07_02.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/F09_06.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/F07_10.txt
Created boundary file: /content/Lipreading_using_Temporal_Convolutional_Networks/annonations/START/F02_03.txt
Created bo

In [12]:
%cd /content/Lipreading_using_Temporal_Convolutional_Networks

# TEST
!python preprocessing/crop_mouth_from_video.py \
  --video-direc /content/Miracl-VC1/ \
  --save-direc datasets/visual_data/ \
  --filename-path test.csv \
  --landmark-direc landmarks/

npz_dir = "/content/Lipreading_using_Temporal_Convolutional_Networks/datasets/visual_data"
output_txt = "/content/Lipreading_using_Temporal_Convolutional_Networks/annotations/test.txt"

with open(output_txt, "w") as f:
    for label in os.listdir(npz_dir):
        label_dir = os.path.join(npz_dir, label)
        if not os.path.isdir(label_dir):
            continue
        for fname in os.listdir(label_dir):
            if fname.endswith(".npz"):
                sample_id = f"{label}/{os.path.splitext(fname)[0]}"
                f.write(sample_id + "\n")

print(f"[ANNOTATIONS] Creato file test.txt con {sum(1 for _ in open(output_txt))} sample")


/content/Lipreading_using_Temporal_Convolutional_Networks
idx: 0 	Processing.	START/F10_09
idx: 1 	Processing.	START/F09_07
idx: 2 	Processing.	START/M08_09
idx: 3 	Processing.	START/M08_01
idx: 4 	Processing.	START/F07_04
idx: 5 	Processing.	START/F07_02
idx: 6 	Processing.	START/F09_06
idx: 7 	Processing.	START/F07_10
idx: 8 	Processing.	START/F02_03
idx: 9 	Processing.	START/M08_02
idx: 10 	Processing.	START/F02_05
idx: 11 	Processing.	START/F09_04
idx: 12 	Processing.	START/M02_03
idx: 13 	Processing.	START/M01_09
idx: 14 	Processing.	START/M08_06
idx: 15 	Processing.	START/F01_03
idx: 16 	Processing.	START/F01_06
idx: 17 	Processing.	START/F09_02
idx: 18 	Processing.	START/F06_01
idx: 19 	Processing.	START/F05_03
idx: 20 	Processing.	START/M07_07
idx: 21 	Processing.	START/F07_06
idx: 22 	Processing.	START/F04_07
idx: 23 	Processing.	START/M08_04
idx: 24 	Processing.	START/F01_01
idx: 25 	Processing.	START/F07_05
idx: 26 	Processing.	START/F04_08
idx: 27 	Processing.	START/F04_09


In [13]:
!CUDA_VISIBLE_DEVICES=0 python main.py \
    --modality video \
    --config-path configs/lrw_resnet18_dctcn.json \
    --model-path models/resnet18_dctcn_video.pth \
    --data-dir datasets/visual_data/ \
    --label-path annotations/test.txt \
    --annonation-direc annonations \
    --labels labels/500WordsSortedList.txt \
    --test

Model and log being saved in: ./train_logs/tcn/2025-08-23T18-47-13
2-norm of the neural network: 81.9003
[CONTROL]:  labels/500WordsSortedList.txt
[DEBUG][video] Inizializzazione dataset
[DEBUG] data_dir = datasets/visual_data/
[DEBUG] label_fp = annotations/test.txt
[FOUND] datasets/visual_data/START/F01_05.npz
[FOUND] datasets/visual_data/START/M04_01.npz
[FOUND] datasets/visual_data/START/F01_07.npz
[FOUND] datasets/visual_data/START/M04_03.npz
[FOUND] datasets/visual_data/START/F04_07.npz
[FOUND] datasets/visual_data/START/M04_07.npz
[FOUND] datasets/visual_data/START/F11_03.npz
[FOUND] datasets/visual_data/START/F08_05.npz
[FOUND] datasets/visual_data/START/F07_03.npz
[FOUND] datasets/visual_data/START/M08_08.npz
[FOUND] datasets/visual_data/START/F02_04.npz
[FOUND] datasets/visual_data/START/M04_08.npz
[FOUND] datasets/visual_data/START/F10_07.npz
[FOUND] datasets/visual_data/START/F09_02.npz
[FOUND] datasets/visual_data/START/F04_10.npz
[FOUND] datasets/visual_data/START/F09_09.